# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Loading Dataset

In [ ]:
from pandas import read_csv
train = pd.read_csv('train_indessa.csv')
# test=pd.read_csv('test_indessa.csv')
loan_df=train.copy()
# loan_df

In [ ]:
loan_df.shape

(304988, 45)

In [ ]:
loan_df.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,batch_enrolled,int_rate,grade,sub_grade,emp_title,...,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,verification_status_joint,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status
0,58189336,14350,14350,14350.0,36 months,,19.19,E,E3,clerk,...,0.0,74.0,INDIVIDUAL,NaN,26th week,0.0,0.0,28699.0,30800.0,0.0
1,70011223,4800,4800,4800.0,36 months,BAT1586599,10.99,B,B4,Human Resources Specialist,...,0.0,NaN,INDIVIDUAL,NaN,9th week,0.0,0.0,9974.0,32900.0,0.0
2,70255675,10000,10000,10000.0,36 months,BAT1586599,7.26,A,A4,Driver,...,0.0,NaN,INDIVIDUAL,NaN,9th week,0.0,65.0,38295.0,34900.0,0.0
3,1893936,15000,15000,15000.0,36 months,BAT4808022,19.72,D,D5,Us office of Personnel Management,...,0.0,NaN,INDIVIDUAL,NaN,135th week,0.0,0.0,55564.0,24700.0,0.0
4,7652106,16000,16000,16000.0,36 months,BAT2833642,10.64,B,B2,LAUSD-HOLLYWOOD HIGH SCHOOL,...,0.0,NaN,INDIVIDUAL,NaN,96th week,0.0,0.0,47159.0,47033.0,0.0


In [ ]:
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304988 entries, 0 to 304987
Data columns (total 45 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   member_id                    304988 non-null  int64  
 1   loan_amnt                    304988 non-null  int64  
 2   funded_amnt                  304988 non-null  int64  
 3   funded_amnt_inv              304988 non-null  float64
 4   term                         304988 non-null  object 
 5   batch_enrolled               256172 non-null  object 
 6   int_rate                     304988 non-null  float64
 7   grade                        304988 non-null  object 
 8   sub_grade                    304988 non-null  object 
 9   emp_title                    287410 non-null  object 
 10  emp_length                   289664 non-null  object 
 11  home_ownership               304988 non-null  object 
 12  annual_inc                   304986 non-null  float64
 13 

# Feature Engineering

In [ ]:
!pip install category_encoders
import category_encoders as ce

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
one_hot_encoded_df = train.copy()
# one_hot_encoded_df0['term'] = one_hot_encoded_df0['term'].map((lambda x: x.strip(' months'))).astype(str).astype(int)
# one_hot_encoded_df = one_hot_encoded_df.rename(columns={'term':'term (in months)'})

encoder=ce.OneHotEncoder(cols= ['term', 'grade', 'home_ownership', 'verification_status', 
                                   'purpose', 'initial_list_status'], use_cat_names=True) 
                                  


# Dropping unnecessary features
one_hot_encoded_df.drop(['member_id', 'batch_enrolled', 'sub_grade', 'emp_title', 'pymnt_plan', 'desc', 
                         'title', 'zip_code', 'addr_state', 'application_type', 'verification_status_joint'], axis=1, inplace= True)

# Numerical Features
one_hot_encoded_df[['total_rev_hi_lim','int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'revol_bal',
         'revol_util', 'total_acc', 'total_rec_int', 'tot_cur_bal', 'total_rec_late_fee', 'recoveries',
         'collection_recovery_fee','collections_12_mths_ex_med','acc_now_delinq', 'funded_amnt_inv']]  =   one_hot_encoded_df[['total_rev_hi_lim','int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'revol_bal',
                    'revol_util', 'total_acc', 'total_rec_int', 'tot_cur_bal', 'total_rec_late_fee', 'recoveries',
                    'collection_recovery_fee','collections_12_mths_ex_med','acc_now_delinq', 'funded_amnt_inv']].fillna(one_hot_encoded_df[
                        ['total_rev_hi_lim','int_rate', 
                            'annual_inc', 'dti', 'delinq_2yrs',
                            'inq_last_6mths', 'open_acc', 'revol_bal',
                            'revol_util', 'total_acc', 'total_rec_int', 
                            'tot_cur_bal', 'total_rec_late_fee', 'recoveries',
                            'collection_recovery_fee','collections_12_mths_ex_med',
                            'acc_now_delinq', 'funded_amnt_inv']].mean()).astype(float). astype(int)




zero_imputation_cols =['delinq_2yrs','inq_last_6mths','collections_12_mths_ex_med','mths_since_last_major_derog','acc_now_delinq']

median_imputation_cols =['emp_length','annual_inc','open_acc','pub_rec','revol_util','total_acc','mths_since_last_delinq','mths_since_last_record','tot_coll_amt',
'tot_cur_bal','total_rev_hi_lim']


one_hot_encoded_df['purpose'].replace(['other', 'major_purchase', 'small_business', 'medical','car', 
                          'moving', 'vacation', 'house', 'wedding', 'renewable_energy', 
                          'educational'], 'Other', inplace=True)
one_hot_encoded_df.replace(['OTHER','NONE','ANY'],'OTHERS',inplace=True)
one_hot_encoded_df['last_week_pay'].replace(to_replace=['th week'], value='', regex=True,inplace= True)
one_hot_encoded_df['last_week_pay'].replace('NA', value= 0,inplace= True)

one_hot_encoded_df['last_week_pay']=pd.to_numeric(one_hot_encoded_df.last_week_pay)
one_hot_encoded_df['term'] = one_hot_encoded_df['term'].map((lambda x: x.strip(' months'))).astype(str)

one_hot_encoded_df['term']=pd.to_numeric(one_hot_encoded_df['term'])
# print(one_hot_encoded_df.info())
one_hot_encoded_df['emp_length'].replace(to_replace=['< 1 year','1 year','2 years','3 years','4 years','5 years',  
                                          '6 years','7 years','8 years','9 years','10+ years'],
                                         value = ['1','1','2','3','4','5','6','7','8','9','10'],inplace=True)
one_hot_encoded_df['emp_length']=pd.to_numeric(one_hot_encoded_df.emp_length)


for col in zero_imputation_cols:
    one_hot_encoded_df[col].fillna(0, inplace=True)

for col in median_imputation_cols:    
    one_hot_encoded_df[col].fillna(one_hot_encoded_df[col].median(), inplace=True)

x = encoder.fit_transform(one_hot_encoded_df)


In [ ]:
x['emp_length'].count

<bound method Series.count of 0          9.0
1          1.0
2          2.0
3         10.0
4         10.0
          ... 
304983     1.0
304984     1.0
304985    10.0
304986     1.0
304987     1.0
Name: emp_length, Length: 304988, dtype: float64>

In [ ]:
x.columns.groupby(x.dtypes)

{int64: ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term_36.0', 'term_60.0', 'int_rate', 'grade_E', 'grade_B', 'grade_A', 'grade_D', 'grade_C', 'grade_F', 'grade_G', 'home_ownership_OWN', 'home_ownership_MORTGAGE', 'home_ownership_RENT', 'home_ownership_OTHERS', 'annual_inc', 'verification_status_Source Verified', 'verification_status_Not Verified', 'verification_status_Verified', 'purpose_debt_consolidation', 'purpose_home_improvement', 'purpose_credit_card', 'purpose_Other', 'purpose_nan', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status_f', 'initial_list_status_w', 'initial_list_status_nan', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_cur_bal', 'total_rev_hi_lim'], float64: ['emp_length', 'mths_since_last_delinq', 'mths_since_last_record', 'pub_rec', 'mths_since_last_major_derog', 'last_week_pay', 'tot_coll_amt', 'loan_status']

In [ ]:
x.dropna(inplace= True)
x = x.reset_index(drop = True)

In [ ]:
x.isna().sum()

loan_amnt                              0
funded_amnt                            0
funded_amnt_inv                        0
term_36.0                              0
term_60.0                              0
int_rate                               0
grade_E                                0
grade_B                                0
grade_A                                0
grade_D                                0
grade_C                                0
grade_F                                0
grade_G                                0
emp_length                             0
home_ownership_OWN                     0
home_ownership_MORTGAGE                0
home_ownership_RENT                    0
home_ownership_OTHERS                  0
annual_inc                             0
verification_status_Source Verified    0
verification_status_Not Verified       0
verification_status_Verified           0
purpose_debt_consolidation             0
purpose_home_improvement               0
purpose_credit_c

# Machine Learning Models

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kfold = KFold(n_splits=10, random_state=7, shuffle=True)

## Splitting Data into Testing and Training set

In [ ]:
final_data=x.copy()
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304987 entries, 0 to 304986
Data columns (total 52 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   loan_amnt                            304987 non-null  int64  
 1   funded_amnt                          304987 non-null  int64  
 2   funded_amnt_inv                      304987 non-null  int64  
 3   term_36.0                            304987 non-null  int64  
 4   term_60.0                            304987 non-null  int64  
 5   int_rate                             304987 non-null  int64  
 6   grade_E                              304987 non-null  int64  
 7   grade_B                              304987 non-null  int64  
 8   grade_A                              304987 non-null  int64  
 9   grade_D                              304987 non-null  int64  
 10  grade_C                              304987 non-null  int64  
 11  grade_F      

In [ ]:
final_data.isna().sum()

loan_amnt                              0
funded_amnt                            0
funded_amnt_inv                        0
term_36.0                              0
term_60.0                              0
int_rate                               0
grade_E                                0
grade_B                                0
grade_A                                0
grade_D                                0
grade_C                                0
grade_F                                0
grade_G                                0
emp_length                             0
home_ownership_OWN                     0
home_ownership_MORTGAGE                0
home_ownership_RENT                    0
home_ownership_OTHERS                  0
annual_inc                             0
verification_status_Source Verified    0
verification_status_Not Verified       0
verification_status_Verified           0
purpose_debt_consolidation             0
purpose_home_improvement               0
purpose_credit_c

In [ ]:
X=final_data.drop('loan_status',axis=1)
X.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,term_36.0,term_60.0,int_rate,grade_E,grade_B,grade_A,grade_D,...,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
count,304987.000000,304987.000000,304987.000000,304987.000000,304987.000000,304987.000000,304987.000000,304987.000000,304987.000000,304987.000000,...,304987.000000,304987.000000,304987.000000,304987.000000,304987.000000,304987.000000,304987.000000,304987.000000,3.049870e+05,3.049870e+05
mean,14747.980734,14734.407286,14694.705217,0.699935,0.300065,12.665261,0.079394,0.286648,0.167138,0.156895,...,0.395459,45.271821,4.749576,0.014184,11.016886,56.949349,0.005056,196.854197,1.394633e+05,3.204436e+04
std,8425.095520,8419.243615,8431.794818,0.458287,0.458287,4.361213,0.270352,0.452197,0.373100,0.363703,...,4.135961,399.298485,59.632118,0.132478,22.060968,44.635838,0.077930,1926.421006,1.468739e+05,3.554495e+04
min,500.000000,500.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,8000.000000,8000.000000,8000.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,0.000000,0.000000,3.219150e+04,1.470000e+04
50%,13000.000000,13000.000000,13000.000000,1.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,48.000000,0.000000,0.000000,9.994000e+04,2.580000e+04
75%,20000.000000,20000.000000,20000.000000,1.000000,1.000000,16.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,83.000000,0.000000,0.000000,1.958040e+05,3.790000e+04
max,35000.000000,35000.000000,35000.000000,1.000000,1.000000,28.000000,1.000000,1.000000,1.000000,1.000000,...,358.000000,29623.000000,7002.000000,16.000000,180.000000,300.000000,5.000000,496651.000000,3.840795e+06,9.999999e+06


In [ ]:
y = final_data['loan_status']
X=final_data.drop('loan_status',axis=1)
y = final_data['loan_status']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=101)

In [ ]:
X_train.isnull().sum()


loan_amnt                              0
funded_amnt                            0
funded_amnt_inv                        0
term_36.0                              0
term_60.0                              0
int_rate                               0
grade_E                                0
grade_B                                0
grade_A                                0
grade_D                                0
grade_C                                0
grade_F                                0
grade_G                                0
emp_length                             0
home_ownership_OWN                     0
home_ownership_MORTGAGE                0
home_ownership_RENT                    0
home_ownership_OTHERS                  0
annual_inc                             0
verification_status_Source Verified    0
verification_status_Not Verified       0
verification_status_Verified           0
purpose_debt_consolidation             0
purpose_home_improvement               0
purpose_credit_c

## Logistic Regression Model

In [ ]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

log_reg= LogisticRegression()
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

### Evaluating Logistic Regression

In [ ]:
y_pred_logreg= log_reg.predict(X_test)
print('Confusion Matrix:', confusion_matrix(y_test, y_pred_logreg))
print('Accuracy Score:', accuracy_score(y_pred_logreg, y_test)*100)

Confusion Matrix: [[69904   204]
 [20850   539]]
Accuracy Score: 76.9894094888357


### Performance Metrics - Cross Validation Evaluation metrics for regression:

#### Mean Absolute Error

In [ ]:
results_MA_1 = cross_val_score(log_reg, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print("Mean Absolute Error: %.3f (%.3f)" % (results_MA_1.mean(), results_MA_1.std()))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Mean Absolute Error: -0.233 (0.002)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


#### Mean Squared Error

In [ ]:
results_MS_1 = cross_val_score(log_reg, X, y, cv=kfold, scoring='neg_mean_squared_error')
print("Mean Squared Error: %.3f (%.3f)" % (results_MS_1.mean(), results_MS_1.std()))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Mean Squared Error: -0.233 (0.002)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier()
dtree.fit(X_train,y_train)

### Evaluating Decision Tree

#### Classification Report

In [ ]:
y_pred_dtree=dtree.predict(X_test)
print(classification_report(y_test,y_pred_dtree))

#### Confusion Matrix

In [ ]:
print('Confusion Matrix:', confusion_matrix(y_test,y_pred_dtree))
print("Accuracy: %.3f%%" % ((accuracy_score(y_pred_dtree, y_test))*100.0))

### Performance Metrics - Cross Validation Evaluation metrics for regression:

#### Mean Absolute Error

In [ ]:
results_MA_2 = cross_val_score(dtree, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print("Mean Absolute Error: %.3f (%.3f)" % (results_MA_2.mean(), results_MA_2.std()))

#### Mean Squared Error

In [ ]:
results_MS_2 = cross_val_score(dtree, X, y, cv=kfold, scoring='neg_mean_squared_error')
print("Mean Squared Error: %.3f (%.3f)" % (results_MS_2.mean(), results_MS_2.std()))

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=600)
rfc.fit(X_train,y_train)

### Evaluating Random Forest Classifier:

In [ ]:
y_pred_rfc=rfc.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print('Classification Report:\n', classification_report(y_test,y_pred_rfc))
print('\n')
print("Confusion matrix \n", confusion_matrix(y_test,y_pred_rfc))
print("Score of Random Forest training Model:",  rfc.score(X_train, y_train))
print("Score of Random Forest testing Model:",  rfc.score(X_test, y_test))
print("Accuracy: %.3f%%" % ((rfc.score(X_test, y_test))*100.0))

### Performance Metrics - Cross Validation Evaluation metrics for regression:

#### Mean Absolute Error

In [ ]:
results_MA_3 = cross_val_score(rfc, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print("Mean Absolute Error: %.3f (%.3f)" % (results_MA_3.mean(), results_MA_3.std()))

#### Mean Squared Error

In [ ]:
results_MS_3 = cross_val_score(rfc, X, y, cv=kfold, scoring='neg_mean_squared_error')
print("Mean Squared Error: %.3f (%.3f)" % (results_MS_3.mean(), results_MS_3.std()))

## K- Nearest Neighbor Classification

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Trying Out N-Neighbors

#### Nearest Neighbor = 1

In [ ]:
knn1 = KNeighborsClassifier(n_neighbors = 1)
knn1.fit(X_train, y_train)

#### Nearest Neighbor = 5

In [ ]:
knn5 = KNeighborsClassifier(n_neighbors = 5)
knn5.fit(X_train, y_train)

#### Nearest Neighbor = 7

In [ ]:
knn7 = KNeighborsClassifier(n_neighbors = 7)
knn7.fit(X_train, y_train)

#### Nearest Neighbor = 10

In [ ]:
knn10 = KNeighborsClassifier(n_neighbors = 10)
knn10.fit(X_train, y_train)

### Evaluation Metrics

In [ ]:
print('KNN =1')
print('Accuracy Score:', (knn1.score(X_test, y_test))*100)

knn1_MA = cross_val_score(knn1, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print("Mean Absolute Error: %.3f (%.3f)" % (knn1_MA.mean(),  knn1_MA.std()))

knn1_MS = cross_val_score(knn1, X, y, cv=kfold, scoring='neg_mean_squared_error')
print("Mean Squared Error: %.3f (%.3f)" % (knn1_MS.mean(), knn1_MS.std()))

knn1_r2 = cross_val_score(knn1, X, y, cv=kfold, scoring='r2')
print("R- squared error: %.3f (%.3f)" % (knn1_r2.mean(), knn1_r2.std()))

In [ ]:
print('KNN = 5')
print('Accuracy Score:', (knn5.score(X_test, y_test))*100)
knn5_MA = cross_val_score(knn5, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print("Mean Absolute Error: %.3f (%.3f)" % (knn5_MA.mean(),  knn5_MA.std()))

knn5_MS = cross_val_score(knn5, X, y, cv=kfold, scoring='neg_mean_squared_error')
print("Mean Squared Error: %.3f (%.3f)" % (knn5_MS.mean(), knn5_MS.std()))

knn5_r2 = cross_val_score(knn5, X, y, cv=kfold, scoring='r2')
print("R- squared error: %.3f (%.3f)" % (knn5_r2.mean(), knn5_r2.std()))

In [ ]:
print('KNN =7')
print('Accuracy Score:', (knn7.score(X_test, y_test))*100)

knn7_MA = cross_val_score(knn7, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print("Mean Absolute Error: %.3f (%.3f)" % (knn7_MA.mean(),  knn7_MA.std()))

knn7_MS = cross_val_score(knn7, X, y, cv=kfold, scoring='neg_mean_squared_error')
print("Mean Squared Error: %.3f (%.3f)" % (knn7_MS.mean(), knn7_MS.std()))

knn7_r2 = cross_val_score(knn7, X, y, cv=kfold, scoring='r2')
print("R- squared error: %.3f (%.3f)" % (knn7_r2.mean(), knn7_r2.std()))

In [ ]:
print('KNN =10')
print('Accuracy Score:', (knn10.score(X_test, y_test))*100)

knn10_MA = cross_val_score(knn10, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print("Mean Absolute Error: %.3f (%.3f)" % (knn10_MA.mean(),  knn10_MA.std()))

knn10_MS = cross_val_score(knn10, X, y, cv=kfold, scoring='neg_mean_squared_error')
print("Mean Squared Error: %.3f (%.3f)" % (knn10_MS.mean(), knn10_MS.std()))

knn10_r2 = cross_val_score(knn10, X, y, cv=kfold, scoring='r2')
print("R- squared error: %.3f (%.3f)" % (knn10_r2.mean(), knn10_r2.std()))

## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {
   "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8,10],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }
gb_clf = GradientBoostingClassifier()

In [ ]:
gb_clf.fit(X_train, y_train)
print(accuracy_score(gb_clf.predict(X_test), y_test))
# print(gb_clf.best_params_)

In [ ]:
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01],
    "min_samples_split": [0.1],
    "min_samples_leaf": [0.1],
    "max_depth":[3],
    "max_features":["log2"],
    "criterion": ["friedman_mse"],
    "subsample":[0.5],
    "n_estimators":[10]
    }
gb_clfb = GridSearchCV(GradientBoostingClassifier(), parameters, cv=10, n_jobs=-1,verbose=1)
gb_clfb.fit(X_train, y_train)
print(gb_clfb.score(X_train, y_train))

In [ ]:
# Predictions:
y_pred_gb=gb_clfb.predict(X_test)

### Evaluating Gradient Boosting Model:

In [ ]:
# For better visualisation:
pd.crosstab(y_test, y_pred_gb, rownames=['Actual'], colnames=['Predicted'], margins=True)

In [ ]:
print('Accuracy Score:', (accuracy_score(y_pred_gb, y_test))*100)

print('Classification Report:\n', classification_report(y_test,y_pred_gb))
print('Confusion Matrix:\n', confusion_matrix(y_pred_gb, y_test))

GB_MA = cross_val_score(gb_clfb, X, y, cv=kfold, scoring='neg_mean_absolute_error')
print("Mean Absolute Error: %.3f (%.3f)" % (GB_MA.mean(),  GB_MA.std()))

GB_MS = cross_val_score(gb_clfb, X, y, cv=kfold, scoring='neg_mean_squared_error')
print("Mean Squared Error: %.3f (%.3f)" % (GB_MS.mean(), GB_MS.std()))

GB_r2 = cross_val_score(gb_clfb, X, y, cv=kfold, scoring='r2')
print("R- squared error: %.3f (%.3f)" % (GB_r2.mean(), GB_r2.std()))